<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/Kacper-W-Kozdon/demos.git
!git clone https://github.com/Kacper-W-Kozdon/ivy.git

Cloning into 'demos'...
remote: Enumerating objects: 3152, done.
remote: Counting objects: 100% (608/608), done.
remote: Compressing objects: 100% (229/229), done.
remote: Total 3152 (delta 440), reused 421 (delta 348), pack-reused 2544
Receiving objects: 100% (3152/3152), 16.26 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (1813/1813), done.
fatal: destination path 'ivy' already exists and is not an empty directory.


In [2]:
!pip install -U -q ivy mlflow datasets>=2.14.5 nlp 2>/dev/null
import ivy

In [12]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np  # For numerical operations and arrays

import warnings  # For handling warnings
warnings.filterwarnings("ignore")  # Ignore warning messages

import torch  # PyTorch library for deep learning
from transformers import AutoModel, AutoTokenizer  # Transformers library for natural language processing
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding  # Transformers components for text processing
from transformers import AutoModelForSequenceClassification  # Transformer model for sequence classification

from nlp import Dataset  # Import custom 'Dataset' class for natural language processing tasks
from imblearn.over_sampling import RandomOverSampler  # For oversampling to handle class imbalance
import datasets  # Import datasets library
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import pipeline  # Transformers library for pipelines
from bs4 import BeautifulSoup  # For parsing HTML content

import matplotlib.pyplot as plt  # For data visualization
import itertools  # For working with iterators
from sklearn.metrics import (  # Import various metrics from scikit-learn
    accuracy_score,  # For calculating accuracy
    roc_auc_score,  # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

from datasets import load_metric  # Import load_metric function to load evaluation metrics

from tqdm import tqdm  # For displaying progress bars
tqdm.pandas()  # Enable progress bars for pandas operations

In [13]:
df = pd.read_csv("/content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv")


In [14]:

df = df.drop_duplicates()
df = df[['comment', 'label']]
df = df[~df['comment'].isnull()]
df = df[~df['label'].isnull()]
print(df.sample(15))

                                                  comment  label
531718  because the american political model can be us...      1
302525  Mary Kate and Ashely: Island Dance Party was c...      0
698658  Egalitarianism is equality in all respects, no...      0
411629  So you'd rather wait until a news article with...      0
978426  Oh dear god how DARE they have a contest where...      1
622068    Have you considered filming it and showing him?      0
346420  One is younger and has made it to the NHL, and...      1
640505  Poor guy in the back trying to see around the ...      0
129036  Why is Booker better at defense than Justise W...      0
841552             you've never joined the thieves guild?      0
676458  **"Backup Exec is not shit"** ^^^^The ^^^^worl...      0
870757  Why don't we all just buy the new iPhone so th...      1
491654                               I know that feel :,(      0
531004  Too late, I've already heard of the accident e...      1
232621  The irony of refe

In [16]:
dataset = Dataset.from_pandas(df)
del df
gc.collect()


151

In [23]:
def get_training_corpus():
  training_corpus = (
    dataset["comment"][i : i + 1000]
    for i in range(0, len(dataset["comment"]), 1000)
  )
  return training_corpus

training_corpus = get_training_corpus()


In [24]:
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)